In [2]:
import json
import sympy as sp

# Polar Pulley Analytical Solutions

This notebook solves analytically for GT-2 and GT-3 pulley tooth profiles. Specifically, it solves for the locations of the relevant arc segment points and centers given the definition of the GT tooth profile. 

![Tooth Profile Diagram](diagrams/pulley_polar_eqns_diagram.svg)
*Tooth profile diagram of GT-#N tooth profiles. Specific dimensions are for a GT-2 pulley with 10 teeth*

## Solution Strategy

Solving for all of the points at once doesn't seem to complete in a reasonable amount of time, so the solution is aproached in several steps.

### Step 1

TODO

In [10]:
# Variables
AX, AY, BX, BY, ABX, ABY, BCX, BCY, RBXY, CX, CY, CDX, CDY, DX, DY = sp.symbols(
    'AX, AY, BX, BY, ABX, ABY, BCX, BCY, RBXY, CX, CY, CDX, CDY, DX, DY', real=True)

In [11]:
# Parameters
RAB, RBC, RCD, b, h, PLD, t, PD = sp.symbols(
    'RAB, RBC, RCD, b, h, PLD, t, PD')
params = [RAB, RBC, RCD, b, h, PLD, t, PD]
vals = [0.555, 1, 0.15, 0.4, 0.75, 0.254,
        10, 6.36619772368]
param_associations = list(zip(params, vals))

In [12]:
# Substitutions
L = sp.symbols("L")
L_sub = PD / 2 - PLD - h + RAB

In [13]:
# Equation set 1. The goal is to solve for BCX, BCY, BX, and BY.
equations_1 = [
    AX + BX,
    AY - BY,
    ABX,
    L - ABY,
    RBXY * sp.sin(2 * b / PD) + BCX,
    RBXY * sp.cos(2 * b / PD) - BCY,
    sp.sqrt(BCX**2 + BCY**2) - RBXY,
    (BX - ABX)**2 + (BY - ABY)**2 - RAB**2,
    (BX - BCX)**2 + (BY - BCY)**2 - RBC**2,
    (BCY - ABY) / (BCX - ABX) - (ABY - BY) / (ABX - BX)
]

In [14]:
res_1 = sp.solve(equations_1, AX, AY, BX, BY, ABX,
                 ABY, BCX, BCY, RBXY, dict=True)

In [16]:
res_1_sol = res_1[1] # Hand-picked solution which matches the drawing
# Substitute back L
for key, val in res_1_sol.items():
    res_1_sol[key] = val.subs(L, L_sub)

In [19]:
N = sp.symbols("N")
N_sub = PD/2 - PLD

In [32]:
# θ is defined from y axis unit vector
# x = r * sin( θ ) y = r * cos( θ ) 

equations_2 = [
    # Treat BCX and BCY as parameters
    (CX - BCX)**2 + (CY - BCY)**2 - RBC**2,
    (CX - CDX)**2 + (CY - CDY)**2 - RCD**2,
    (BCX - CX) / (BCY - CY) - (CX - CDX) / (CY - CDY),
#     DX**2 + DY**2 - N**2,
    
    CDX - DX*(N-RCD)/N,
    CDY - DY*(N-RCD)/N
]

In [33]:
res_2 = sp.solve(equations_2, CX, CY, DX, DY, CDX, CDY, dict=True)

In [39]:
res_1_sol

{AX: (-2*RAB**2*(PD/2 - PLD + RAB - h)*sin(b/PD)**2 + RAB**2*(PD/2 - PLD + RAB - h) + RAB**2*sqrt(RAB**2 - 2*RAB*RBC + RBC**2 + 4*(PD/2 - PLD + RAB - h)**2*sin(b/PD)**4 - 4*(PD/2 - PLD + RAB - h)**2*sin(b/PD)**2) - 2*RAB*RBC*(PD/2 - PLD + RAB - h)*sin(b/PD)**2 + RAB*RBC*(PD/2 - PLD + RAB - h) + RAB*RBC*sqrt(RAB**2 - 2*RAB*RBC + RBC**2 + 4*(PD/2 - PLD + RAB - h)**2*sin(b/PD)**4 - 4*(PD/2 - PLD + RAB - h)**2*sin(b/PD)**2) + 256*(PD/2 - PLD + RAB - h)**3*sin(b/PD)**18 - 1152*(PD/2 - PLD + RAB - h)**3*sin(b/PD)**16 + 2112*(PD/2 - PLD + RAB - h)**3*sin(b/PD)**14 + 256*(PD/2 - PLD + RAB - h)**3*sin(b/PD)**12*cos(b/PD)**6 - 2016*(PD/2 - PLD + RAB - h)**3*sin(b/PD)**12 - 384*(PD/2 - PLD + RAB - h)**3*sin(b/PD)**10*cos(b/PD)**6 + 1056*(PD/2 - PLD + RAB - h)**3*sin(b/PD)**10 + 192*(PD/2 - PLD + RAB - h)**3*sin(b/PD)**8*cos(b/PD)**6 - 288*(PD/2 - PLD + RAB - h)**3*sin(b/PD)**8 - 32*(PD/2 - PLD + RAB - h)**3*sin(b/PD)**6*cos(b/PD)**6 + 32*(PD/2 - PLD + RAB - h)**3*sin(b/PD)**6)*sin(2*b/PD)/(RAB**2